In [13]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

con = sqlite3.connect('database.sqlite') 

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 5000""", con) 
def partition(x):
    if x < 3:
        return 0
    return 1

actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

from tqdm import tqdm
from bs4 import BeautifulSoup

preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

i=0
list_of_sentance=[]
for sentance in preprocessed_reviews:
    list_of_sentance.append(sentance.split())
    
w2v_model=Word2Vec(list_of_sentance,min_count=5,vector_size=50, workers=4)
tst=w2v_model.wv.most_similar('tasty',topn=50)
#tst=w2v_model.wv.most_similar('distasteful',topn=50)

tt=w2v_model.wv.most_similar('good',topn=50)
ls=w2v_model.wv.most_similar('bad',topn=25)
ls.extend(w2v_model.wv.most_similar('terrible',topn=25))

ts=w2v_model.wv.most_similar('superior',topn=50)
#ls.extend(w2v_model.wv.most_similar('mediocre',topn=50))

ts=w2v_model.wv.most_similar('damage',topn=50)

#good
k=0
for i in list_of_sentance:
    for j in tt:
        if j[0] in i:
            k+=1
            break

#bad and terrible
c=0
for i in list_of_sentance:
    for j in ls:
        if j[0] in i:
            c+=1
            break
        
#tasty
x=0
for i in list_of_sentance:
    for j in tst:
        if j[0] in i:
            x+=1
            break
            
#bad and terrible taste
z=0
for i in list_of_sentance:
    for j in ls:
         if j[0] in i:
            if 'taste' in i:
                z+=1
                break

#superior
y=0
for i in list_of_sentance:
    for j in ts:
        if j[0] in i:
                y+=1
                break

#not superior

y=len(list_of_sentance)
for i in list_of_sentance:
    for j in ts:
        if j[0] in i:
                y-=1
                break

                
d=str(k)+' - good '+str(c)'- bad'
da=str(k)+' - tasty '+str(c)'- not tasty'
dat=str(k)+' - superior '+str(c)'- not superior'


100%|████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:02<00:00, 1775.80it/s]


In [22]:
# Total words
f=0
for i in list_of_sentance:
    f+=1
print(f)

4986


In [15]:
#good
k=0
for i in list_of_sentance:
    for j in tt:
        if j[0] in i:
            k+=1
            break
print(k)

4202


In [16]:
#bad and terrible
c=0
for i in list_of_sentance:
    for j in ls:
        if j[0] in i:
            c+=1
            break
print(c)

2428


In [17]:
#tasty
x=0
for i in list_of_sentance:
    for j in tst:
        if j[0] in i:
            x+=1
            break
print(x)

2866


In [23]:
#bad and terrible taste
z=0
for i in list_of_sentance:
    for j in ls:
         if j[0] in i:
            if 'taste' in i:
                z+=1
                break
            
            
print(z)

630


In [19]:
#superior
y=0
for i in list_of_sentance:
    for j in ts:
        if j[0] in i:
                y+=1
                break
print(y)

1450


In [30]:
y=len(list_of_sentance)
for i in list_of_sentance:
    for j in ts:
        if j[0] in i:
                y-=1
                break
print(y)

3536
